## Setup

In [1]:
import pandas as pd
data_dir = "../data/"
output_dir = "../output/"

In [2]:
thresh = 0.00001 # 1e-5
species = pd.read_csv(data_dir + "taxonomy.csv", index_col = 0).T
pathways = pd.read_csv(data_dir + "pathways.csv", index_col = 0).T
X = (pd.concat([species, pathways], axis = 1) > thresh) * 1
X.head()

,s__Abiotrophia_defectiva,s__Acidaminococcus_fermentans,s__Acidaminococcus_intestini,s__Actinomyces_graevenitzii,s__Actinomyces_odontolyticus,s__Actinomyces_oris,s__Actinomyces_turicensis,s__Actinomyces_viscosus,s__Adlercreutzia_equolifaciens,s__Aggregatibacter_segnis,...,TRNA-CHARGING-PWY: tRNA charging,TRPSYN-PWY: L-tryptophan biosynthesis,TYRFUMCAT-PWY: L-tyrosine degradation I,UBISYN-PWY: superpathway of ubiquinol-8 biosynthesis (prokaryotic),UDPNACETYLGALSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis II,UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,URDEGR-PWY: superpathway of allantoin degradation in plants,URSIN-PWY: ureide biosynthesis,VALDEG-PWY: L-valine degradation I,VALSYN-PWY: L-valine biosynthesis
SAMD00036192,0,0,0,1,1,0,0,0,0,0,...,1,1,0,1,0,1,0,0,0,1
SAMD00036193,0,0,0,1,0,0,0,0,1,0,...,1,1,0,0,0,1,0,0,0,1
SAMD00036194,0,0,0,0,0,0,0,0,1,0,...,1,1,0,0,0,1,0,0,0,1
SAMD00036197,0,0,0,1,1,0,0,1,1,0,...,1,1,0,0,0,1,0,0,0,1
SAMD00036204,0,0,0,1,1,1,0,0,1,0,...,1,1,0,1,0,1,1,0,0,1


In [3]:
y = pd.read_csv(data_dir + "isHealthy.csv", index_col=0).T
y.head()

,isHealthy
SAMD00036192,True
SAMD00036193,True
SAMD00036194,False
SAMD00036197,True
SAMD00036204,True


## Use Lasso Regression to select features

In [4]:
from sklearn.linear_model import LogisticRegression
import numpy as np

logreg = LogisticRegression(C=0.1, penalty='l1',
fit_intercept=False, max_iter=500, random_state=42,
                    solver='saga', n_jobs=-1)
clf = logreg.fit(X, np.ravel(y))

In [26]:
coefficients = pd.DataFrame(X.columns, columns=['feature'])
coefficients['coef'] = clf.coef_[0]
coefficients.head()

,feature,coef
0,s__Abiotrophia_defectiva,0.000000
1,s__Acidaminococcus_fermentans,-0.113528
2,s__Acidaminococcus_intestini,-0.099617
3,s__Actinomyces_graevenitzii,0.000000
4,s__Actinomyces_odontolyticus,0.252240


In [27]:
div = 0.0001
pivot = pd.concat([X[y['isHealthy']].mean(), X[~y['isHealthy']].mean()], axis = 1)
pivot.columns = ['healthy', 'not']
pivot['diff'] = pivot['healthy'] - pivot['not']
pivot['foldh'] = (pivot['healthy'] + div) / (pivot['not'] + div)
pivot['foldn'] = (pivot['not'] + div) / (pivot['healthy'] + div)
pivot['weight'] = 0
pivot.loc[pivot['diff'] > 0, 'weight'] = np.log(pivot[pivot['diff'] > 0]['foldh'])
pivot.loc[pivot['diff'] < 0, 'weight'] = np.log(pivot[pivot['diff'] < 0]['foldn'])
pivot['weight2'] = (pivot['weight'] * pivot['diff'])
pivot

,healthy,not,diff,foldh,foldn,weight,weight2
s__Abiotrophia_defectiva,0.006829,0.049679,-0.042850,0.139187,7.184578,1.971937,-0.084498
s__Acidaminococcus_fermentans,0.040212,0.078317,-0.038104,0.514079,1.945225,0.665378,-0.025354
s__Acidaminococcus_intestini,0.080046,0.115722,-0.035676,0.691973,1.445144,0.368209,-0.013136
s__Actinomyces_graevenitzii,0.045144,0.122735,-0.077591,0.368332,2.714942,0.998770,-0.077496
s__Actinomyces_odontolyticus,0.089909,0.155465,-0.065556,0.578595,1.728324,0.547152,-0.035869
...,...,...,...,...,...,...,...
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,0.991275,0.990064,0.001210,1.001222,0.998779,0.001222,0.000001
URDEGR-PWY: superpathway of allantoin degradation in plants,0.295903,0.443016,-0.147113,0.668003,1.496998,0.403462,-0.059354
URSIN-PWY: ureide biosynthesis,0.003414,0.004091,-0.000677,0.838491,1.192618,0.176151,-0.000119
VALDEG-PWY: L-valine degradation I,0.006449,0.003507,0.002942,1.815822,0.550715,0.596539,0.001755


In [28]:
coefficients.index = coefficients['feature']
del coefficients['feature']
coefficients

,coef
feature,
s__Abiotrophia_defectiva,0.000000
s__Acidaminococcus_fermentans,-0.113528
s__Acidaminococcus_intestini,-0.099617
s__Actinomyces_graevenitzii,0.000000
s__Actinomyces_odontolyticus,0.252240
...,...
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,0.000000
URDEGR-PWY: superpathway of allantoin degradation in plants,0.000000
URSIN-PWY: ureide biosynthesis,0.000000


In [29]:
coefficients['weight2'] = pivot['weight2']
coefficients

,coef,weight2
feature,,
s__Abiotrophia_defectiva,0.000000,-0.084498
s__Acidaminococcus_fermentans,-0.113528,-0.025354
s__Acidaminococcus_intestini,-0.099617,-0.013136
s__Actinomyces_graevenitzii,0.000000,-0.077496
s__Actinomyces_odontolyticus,0.252240,-0.035869
...,...,...
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,0.000000,0.000001
URDEGR-PWY: superpathway of allantoin degradation in plants,0.000000,-0.059354
URSIN-PWY: ureide biosynthesis,0.000000,-0.000119


In [30]:
coefficients['abscoef'] = abs(coefficients['coef'])
coefficients

,coef,weight2,abscoef
feature,,,
s__Abiotrophia_defectiva,0.000000,-0.084498,0.000000
s__Acidaminococcus_fermentans,-0.113528,-0.025354,0.113528
s__Acidaminococcus_intestini,-0.099617,-0.013136,0.099617
s__Actinomyces_graevenitzii,0.000000,-0.077496,0.000000
s__Actinomyces_odontolyticus,0.252240,-0.035869,0.252240
...,...,...,...
UDPNAGSYN-PWY: UDP-N-acetyl-D-glucosamine biosynthesis I,0.000000,0.000001,0.000000
URDEGR-PWY: superpathway of allantoin degradation in plants,0.000000,-0.059354,0.000000
URSIN-PWY: ureide biosynthesis,0.000000,-0.000119,0.000000


In [31]:
sorted_df = coefficients.sort_values('abscoef', ascending=False)
sorted_df

,coef,weight2,abscoef
feature,,,
s__Bifidobacterium_angulatum,0.642635,0.213397,0.642635
s__Peptostreptococcus_stomatis,-0.618613,-0.235289,0.618613
s__Solobacterium_moorei,-0.614407,-0.364616,0.614407
s__Lactobacillus_acidophilus,0.516800,0.049587,0.516800
s__Clostridium_bolteae,-0.501387,-0.248070,0.501387
...,...,...,...
FERMENTATION-PWY: mixed acid fermentation,0.000000,-0.003179,0.000000
FOLSYN-PWY: superpathway of tetrahydrofolate biosynthesis and salvage,0.000000,-0.001264,0.000000
FUCCAT-PWY: fucose degradation,0.000000,-0.000169,0.000000


In [72]:
sorted_df['match_sign'] = (sorted_df['coef'] > 0) == (sorted_df['weight2'] > 0)
nonzero = sorted_df[sorted_df['abscoef'] > 0]
above_thresh = nonzero[nonzero['abscoef'] > 0.1]
above_thresh

,coef,weight2,abscoef,match_sign
feature,,,,
s__Bifidobacterium_angulatum,0.642635,0.213397,0.642635,True
s__Peptostreptococcus_stomatis,-0.618613,-0.235289,0.618613,True
s__Solobacterium_moorei,-0.614407,-0.364616,0.614407,True
s__Lactobacillus_acidophilus,0.516800,0.049587,0.516800,True
s__Clostridium_bolteae,-0.501387,-0.248070,0.501387,True
...,...,...,...,...
PWY-6629: superpathway of L-tryptophan biosynthesis,-0.114855,-0.110203,0.114855,True
s__Acidaminococcus_fermentans,-0.113528,-0.025354,0.113528,True
s__Bacteroides_clarus,0.112191,0.016824,0.112191,True


In [73]:
feature_list = list(above_thresh.index)
feature_list

['s__Bifidobacterium_angulatum',
 's__Peptostreptococcus_stomatis',
 's__Solobacterium_moorei',
 's__Lactobacillus_acidophilus',
 's__Clostridium_bolteae',
 'P164-PWY: purine nucleobases degradation I (anaerobic)',
 's__Subdoligranulum_variabile',
 'HISDEG-PWY: L-histidine degradation I',
 'PWY66-389: phytol degradation',
 's__Bifidobacterium_catenulatum',
 'FUC-RHAMCAT-PWY: superpathway of fucose and rhamnose degradation',
 's__Lactobacillus_mucosae',
 's__Faecalibacterium_prausnitzii',
 's__Holdemania_filiformis',
 's__Ruminococcaceae_bacterium_D16',
 's__Oxalobacter_formigenes',
 's__Granulicatella_adiacens',
 's__Bacteroides_caccae',
 's__Streptococcus_gordonii',
 's__Clostridium_methylpentosum',
 's__Butyricimonas_synergistica',
 's__Prevotella_timonensis',
 's__Eubacterium_eligens',
 's__Eubacterium_cylindroides',
 's__Clostridium_celatum',
 's__Actinomyces_odontolyticus',
 's__Bacteroidales_bacterium_ph8',
 's__Haemophilus_parainfluenzae',
 's__Clostridium_bartlettii',
 's__Lact

In [74]:
logreg = LogisticRegression(C=0.1, penalty='l1',
fit_intercept=False, max_iter=500, random_state=42,
                    solver='saga', n_jobs=-1)
clf = logreg.fit(X[feature_list], np.ravel(y))

In [76]:
coefficients = pd.DataFrame(feature_list, columns=['feature'])
coefficients['coef'] = clf.coef_[0]
coefficients.head()

,feature,coef
0,s__Bifidobacterium_angulatum,0.628954
1,s__Peptostreptococcus_stomatis,-0.644409
2,s__Solobacterium_moorei,-0.616561
3,s__Lactobacillus_acidophilus,0.505078
4,s__Clostridium_bolteae,-0.510624


In [78]:
coefficients.index = coefficients['feature']
del coefficients['feature']
coefficients['weight2'] = above_thresh['weight2']
coefficients

,coef,weight2
feature,,
s__Bifidobacterium_angulatum,0.628954,0.213397
s__Peptostreptococcus_stomatis,-0.644409,-0.235289
s__Solobacterium_moorei,-0.616561,-0.364616
s__Lactobacillus_acidophilus,0.505078,0.049587
s__Clostridium_bolteae,-0.510624,-0.248070
...,...,...
PWY-6629: superpathway of L-tryptophan biosynthesis,-0.056182,-0.110203
s__Acidaminococcus_fermentans,-0.200940,-0.025354
s__Bacteroides_clarus,0.120466,0.016824


In [80]:
coefficients['match'] = (coefficients['coef'] > 0) == (coefficients['weight2'] > 0)
coefficients['match'].sum()

65